# 03-2 트랜스포머를 사용한 텍스트 생성

<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/rickiepark/hm-dl/blob/main/03-2-llama3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

코랩에서 이 노트북을 실행하려면 High-RAM CPU 런타임을 사용해야 합니다.

In [1]:
import transformers

transformers.__version__

'4.41.0'

## LLaMa-3

In [2]:
from huggingface_hub import notebook_login

notebook_login()

### 라마-3 구조 살펴 보기

In [5]:
from transformers import pipeline, set_seed

In [6]:
llama3_pipe = pipeline("text-generation", model="meta-llama/Meta-Llama-3-8B")

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
llama3_pipe.model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

In [8]:
pip install torchinfo

In [9]:
from torchinfo import summary

summary(llama3_pipe.model)

Layer (type:depth-idx)                                  Param #
LlamaForCausalLM                                        --
├─LlamaModel: 1-1                                       --
│    └─Embedding: 2-1                                   525,336,576
│    └─ModuleList: 2-2                                  --
│    │    └─LlamaDecoderLayer: 3-1                      218,112,000
│    │    └─LlamaDecoderLayer: 3-2                      218,112,000
│    │    └─LlamaDecoderLayer: 3-3                      218,112,000
│    │    └─LlamaDecoderLayer: 3-4                      218,112,000
│    │    └─LlamaDecoderLayer: 3-5                      218,112,000
│    │    └─LlamaDecoderLayer: 3-6                      218,112,000
│    │    └─LlamaDecoderLayer: 3-7                      218,112,000
│    │    └─LlamaDecoderLayer: 3-8                      218,112,000
│    │    └─LlamaDecoderLayer: 3-9                      218,112,000
│    │    └─LlamaDecoderLayer: 3-10                     218,112,000
│    │    └

### 라마-3로 텍스트 생성하기

In [10]:
llama3_pipe.model.generation_config.pad_token_id = llama3_pipe.tokenizer.eos_token_id

In [11]:
set_seed(42)
llama3_pipe('stay hungry, stay', max_length=20, truncation=True)

[{'generated_text': 'stay hungry, stay alive," he said. "That\'s what I\'m doing right now."\nH'}]

In [12]:
llama3_pipe.model.generation_config

GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": 128001,
  "max_length": 4096,
  "pad_token_id": 128001,
  "temperature": 0.6,
  "top_p": 0.9
}

In [13]:
set_seed(42)
llama3_pipe('봄이 오면', max_length=20, truncation=True)

[{'generated_text': '봄이 오면, 그때 그때 맞는 옷을 입고, 그때 그'}]